# Load Packages

In [ ]:
import pandas as pd
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt
import scrublet as scr
import sklearn.metrics
import re
import harmonypy as hm

# Sequencing Metrics per Sample

In [ ]:
#The sequencing metrics file output from cellranger count is read in for each sample and formatted to give an overview to sample quality 
samplelist = ['11_001','11_002','12_003','12_007','12_008','12_010','14_012','16_013','16_014','AS1','AS10','AS11','AS2','AS3','AS4','AS5','AS6','AS7','AS8','AS9','R11','R12','R4','R6','R7','R8']

tocat = []
for i in samplelist:
    df = pd.read_csv('./' + i + '_metrics.csv')
    df['sample'] = i
    tocat.append(df)
metrics = pd.concat(tocat)

metrics.to_csv('./sample_metrics.csv')

In [ ]:
#Plot of summary statistics from the sequencing metric dataframe to give an overview of sample quality
pd.options.display.max_columns = 100
f, ax = plt.subplots(4,5,figsize=(15,10))
f.subplots_adjust(hspace = 0.4, wspace = 0.5)
xs = [0,1,2,3,4] * 4
ys = [i for j in [[x]*5 for x in range(4)] for i in j]
for xc,yc,met in zip(xs, ys, metrics.columns[:19]):
    boxdict = ax[yc][xc].boxplot([float(re.sub('%','',re.sub(',','',str(x)))) for x in metrics[met]])
    fliers = boxdict['fliers']
    ax[yc][xc].set_title('\n'.join([' '.join(list(z)) for z in np.array_split(met.split(' '), 2)]),
                         fontsize = 8)
    for j in range(len(fliers)):
        yfliers = boxdict['fliers'][j].get_ydata()
        xfliers = boxdict['fliers'][j].get_xdata()
        if len(yfliers) == 0:
            continue
        side = 'left'
        ct = -1
        for xfli, yfli in zip(xfliers, yfliers):
            if metrics[[float(re.sub('%','',re.sub(',','',str(x)))) for x in metrics[met]] == yfli].shape[0] == 1:
                label = metrics[[float(re.sub('%','',re.sub(',','',str(x)))) for x in metrics[met]] == yfli]['sample'].tolist()[0]
            else:
                ct += 1
                label = metrics[[float(re.sub('%','',re.sub(',','',str(x)))) for x in metrics[met]] == yfli]['sample'].tolist()[ct]
            label = '_'.join(label.split('_')[:-1])
            if side == 'left':
                ax[yc][xc].text(xfli - 0.03, yfli + 0.03, label, fontsize = 8, ha = 'right')
                side = 'right'
            else:
                ax[yc][xc].text(xfli + 0.03, yfli + 0.03, label, fontsize = 8, ha = 'left')
                side = 'left'

In [ ]:
!mkdir ./umi_curve_data/

In [ ]:
#Calculation of UMI decay curve for each sample. The raw 10X cellranger .h5 and the filtered cellbender .h5 are required
for s in samplelist:
    print(s)
    cr = sc.read_10x_h5('./' +s+'_raw_cr.h5') # The raw 10X cellranger h5
    cr.var_names_make_unique()
    cb = sc.read_10x_h5('./' +s+'_out_filtered.h5') #filtered cellbender h5
    cr.var['gene_ids'] = cr.var.index.tolist()
    
    cr.var['mt'] = False
    for i in range(len(cr.var['gene_ids'])):
        cr.var['mt'][i] = cr.var['gene_ids'].iloc[i].startswith('MT-')

    sc.pp.calculate_qc_metrics(cr, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
    
    cr = cr[cr.obs['total_counts'] != 0]
    
    forplot = cr.obs.copy()
    forplot.sort_values('total_counts', ascending=False, inplace=True)
    forplot['rank'] = range(1, forplot.shape[0]+1, 1)
    forplot['cell'] = forplot.index.isin(cb.obs.index)
    forplot.to_csv('./umi_curve_data/'+s+'_umi_curve.txt', sep='\t')

In [ ]:
# One example plot of the UMI decay curve showing cell calls with an overlay of mitochondrial reads. This plot is used to check sample quality
s= 'AS11'
toplot = pd.read_csv('./umi_curve_data/'+s+'_umi_curve.txt', sep='\t')
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.loglog(toplot['rank'], toplot['total_counts'], rasterized=True)
ax1.scatter(toplot[toplot['cell']]['rank'], toplot[toplot['cell']]['total_counts'], rasterized=True, color = 'red', alpha = 1, s = 4)
ax2.scatter(toplot['rank'], toplot['pct_counts_mt'], s = 1, alpha = .2, rasterized=True, color = 'green')
ax1.axvline(1000, color='black', linestyle = '--', linewidth = 0.5)
ax1.axvline(5000, color='black', linestyle = '--', linewidth = 0.5)
ax1.axvline(10000, color='black', linestyle = '--', linewidth = 0.5)
ax1.axvline(20000, color='black', linestyle = '--', linewidth = 0.5)
ax1.axvline(40000, color='black', linestyle = '--', linewidth = 0.5)
ax1.axvline(70000, color='black', linestyle = '--', linewidth = 0.5)
ax1.axhline(100000, color='black', linestyle = '--', linewidth = 0.5)
ax1.axhline(50000, color='black', linestyle = '--', linewidth = 0.5)
ax1.axhline(10000, color='black', linestyle = '--', linewidth = 0.5)
ax1.axhline(1000, color='black', linestyle = '--', linewidth = 0.5)
ax1.axhline(500, color='black', linestyle = '--', linewidth = 0.5)
ax1.axhline(100, color='black', linestyle = '--', linewidth = 0.5)
ax1.axhline(10, color='black', linestyle = '--', linewidth = 0.5)
ax1.set_title(s)
ax1.set_xlabel('Ranked Cell Barcode')
ax1.set_ylabel('UMI count')
ax2.set_ylabel('Percent MT Genes')

# Data Processing

In [ ]:
samplelist = ['11_001','11_002','12_003','12_007','12_008','12_010','14_012','16_013','16_014','AS1','AS10','AS11','AS2','AS3','AS4','AS5','AS6','AS7','AS8','AS9']

counter = 0
for s in samplelist:
    print(s)
    #Read in the filtered cellbender h5 file
    adata1 = sc.read_10x_h5('./' +s+'_out_filtered.h5')
    adata1.var_names_make_unique()
    
    #Scrublet - calculate doublet scores for each sample
    scrub = scr.Scrublet(adata1.X)
    adata1.obs['doublet_scores'], adata1.obs['predicted_doublets'] = scrub.scrub_doublets()
    scrub.plot_histogram()
        
    #Scrinvex - load in the scrinvex output tsv and calculate the exon/intron ratio
    scrinvex = pd.read_csv('./' +s+'.scrinvex.tsv',sep='\t')
    scrinvex_collapse=scrinvex.groupby("barcode").sum()
    scrinvex = None
    
    scrinvex_collapse['exon_ratio'] = scrinvex_collapse['exons'] / scrinvex_collapse[['introns','junctions','exons']].sum(1)
    adata1.obs['barcode'] = adata1.obs.index.copy()
    for i in range(len(adata1)):
        adata1.obs['barcode'].iloc[i] = adata1.obs['barcode'].iloc[i]
    newobs = adata1.obs.merge(scrinvex_collapse, left_on='barcode', right_on='barcode', how='left')
    adata1.obs['exon_ratio'] = newobs['exon_ratio'].tolist()
    q3_scrinvex = np.nanpercentile(adata1.obs['exon_ratio'],75)
    q1_scrinvex = np.nanpercentile(adata1.obs['exon_ratio'],25)
    iqr_scrinvex = q3_scrinvex - q1_scrinvex
    cut_scrinvex = q3_scrinvex + 1 * iqr_scrinvex
    adata1.obs['fail_exonratio'] = [True if x > cut_scrinvex else False for x in adata1.obs['exon_ratio']]
    scrinvex_collapse = None
    
    #Apply sample name to data before combining it with the raw anndata
    adata1.obs['batches']=s
    
    if counter==0:
        adata = adata1
    else:
        adata = adata.concatenate(adata1,join='outer')
        
    counter+=1
    
adata.write_h5ad('./AS_rna_raw.h5ad')

In [ ]:
#Adjustment of doublet threshold. A threshold of 0.25 was selected based on the predicted doublet score plots generated above
adata.obs['predicted_doublets'] = False

ind = adata.obs['doublet_scores']>0.25
adata.obs['predicted_doublets'][ind] = True

In [ ]:
#filter low quality cells by number of mapped genes
sc.pp.filter_cells(adata, min_genes=200)

In [ ]:
#Define mitochrondrial genes
adata.var['mt'] = False
for x in range(len(adata.var['gene_ids'])):
    adata.var['mt'][x] = adata.var['gene_ids'].index[x].startswith('MT-')

#Check levels of mitochondrial RNA as QC metric. If mtRNA are too high, could be a low quality cell
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)


In [ ]:
#Removing low quality cells
adata = adata[adata.obs.pct_counts_mt < 5, :] #Removes cells with percentage of MT reads >5%
adata = adata[adata.obs.fail_exonratio == False, :] #Removes cells with a high exon/intron ratio
adata = adata[adata.obs.predicted_doublets == False, :] #Removes cells with a high doublet score


In [ ]:
#Normalize read counts per cell
sc.pp.normalize_total(adata, target_sum=1e4)

#Logarithmize the data
sc.pp.log1p(adata)

#Identify highly variable genes
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

#Remove MT genes. They are helpful for QC, but not desired for visualization and downstream analyses
adata = adata[:,~adata.var.mt]

#Set aside raw data with full list of genes for later use
adata.raw = adata

#Filter for highly variable genes, needed for regressing out effects, scaling, batch correction
adata = adata[:, adata.var.highly_variable]

#Regress out effects of total cell counts/cell and mtRNA
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])

#Scale data to unit variance
sc.pp.scale(adata, max_value=10)

In [ ]:
#Initial principle component analysis for batch correction
sc.tl.pca(adata, svd_solver='arpack')

#Define variable to use for batch correction. Here it is individual samples
vars_use = ['batches']

#Run batch correction. Builds converging model
ho = hm.run_harmony(adata.obsm['X_pca'][:,0:49], adata.obs, vars_use)

#Pull batch-corrected values from model
res = ho.Z_corr
res=res.T

#Add batch-corrected values to anndata
adata.obsm["X_pca"] = res

In [ ]:
#Neighbor calculation using batch-corrected PCA values
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

#Generate umap of neighbors
sc.tl.umap(adata)

In [ ]:
#Leiden clustering on Umap
sc.tl.leiden(adata, key_added='leiden',resolution=0.2)

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.settings.set_figure_params(dpi=80,dpi_save=300, facecolor='white',vector_friendly=False)

sc.pl.umap(adata, color=['leiden'])

In [ ]:
adata_full = sc.AnnData(X=adata.raw.X, obs=adata.obs, var=adata.raw.var, uns=adata.uns, obsm=adata.obsm, varm=None, layers=None, raw=adata.raw, dtype='float32', shape=None, filename=None, filemode=None, asview=False)
adata_full.write('./AS_full_rna_processed.h5ad')

# AUC for Cell type Assignment

In [ ]:
adata = sc.read_h5ad('./AS_full_rna_processed.h5ad')

In [ ]:
#Necessary reformatting of count matrixx
adata.X = adata.X.toarray()

adata.uns['log1p']["base"] = None

#Define the number of groups
c=10

In [ ]:
#find differential genes with wilcoxon rank sum (for calculating some downstream variables)
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon',pts=True, reference='rest',use_raw=True)

In [ ]:
#Build a table with scores and genes
result = adata.uns['rank_genes_groups']
groups = result['names'].dtype.names
expression_df=pd.DataFrame(
    {str(group) + '_' + key: result[key][group]
    for group in groups for key in ['names','pvals_adj', 'logfoldchanges','pts']})

In [ ]:
#Fix order of pts to match group-specific gene order
for i in range(3,len(expression_df.columns),4):
    pts_dict=pd.Series(expression_df[expression_df.columns[i]],index=expression_df.index).to_dict()
    for n in range(0,len(expression_df[expression_df.columns[i-3]])):
        expression_df[expression_df.columns[i]][n]=pts_dict[expression_df[expression_df.columns[i-3]][n]]

In [ ]:
#Calculate AUC for all groups
auc_scores = pd.DataFrame(columns = range(c),index = range(len(adata.var)))
for L in list(np.unique(adata.obs['leiden'])):
    adata.obs['AUC_group'] = 0
    for x in range(len(adata.obs)):
        if adata.obs['leiden'][x] == L:
            adata.obs['AUC_group'][x] = 1
            
    for col in range(len(adata.var)):
        auc_scores.loc[col,L] = sklearn.metrics.roc_auc_score(adata.obs['AUC_group'], adata.X[:,col])


In [ ]:
#Incorporate AUC into info table
auc_df = pd.DataFrame(columns = range(c*4),index = range(len(adata.var)))
for i in range(c):
    log_dict=pd.Series(data=list(expression_df[expression_df.columns[i*4+2]]),index=list(expression_df[expression_df.columns[i*4]])).to_dict()
    pts_dict=pd.Series(data=list(expression_df[expression_df.columns[i*4+3]]),index=list(expression_df[expression_df.columns[i*4]])).to_dict()
    auc_df[auc_df.columns[i*4]] = adataC.var.index
    auc_df[auc_df.columns[i*4+1]] = auc_scores[auc_scores.columns[i]]
    for n in range(0,len(adataC.var)):
        auc_df[auc_df.columns[i*4+2]][n]=log_dict[auc_df[auc_df.columns[i*4]][n]]
        auc_df[auc_df.columns[i*4+3]][n]=pts_dict[auc_df[auc_df.columns[i*4]][n]]

In [ ]:
#Remove genes that fail significance threshold
for i in range(0,c*4,4):
    for g in range(len(auc_df[i])):
        if auc_df[i+1][g]<0.7 or auc_df[i+2][g]<0.6:
            for i_loop in range(i,i+4):
                auc_df[i_loop][g] = np.nan

In [ ]:
for i in range(0,c*4,4):
    auc_subgroup = auc_df[auc_df.columns[i:i+4]]
    auc_subgroup.sort_values(by=[auc_subgroup.columns[1],auc_subgroup.columns[2],auc_subgroup.columns[3]],ascending=False,inplace=True)
    for j in range(0,4):
        auc_df[i+j] = list(auc_subgroup[auc_subgroup.columns[j]])

In [ ]:
#Name dataframe columns
col_names=[]
for i in list(range(c)):
    col_names.append(str(i)+'_name')
    col_names.append(str(i)+'_AUC')
    col_names.append(str(i)+'_lfc')
    col_names.append(str(i)+'_pts')
    
auc_df.columns = col_names

#Print dataframe to file and upload to the cloud
auc_df.to_csv('AUC_filtered.csv')